### A Practical Guide to Building a Conversational Chat Bot

#### **By:** <b>Imonmion Emmanuel</b>

#### For this Notebook we would using some Python Packages

__Packages we would use For This Notebook:__
1. Json
2. Numpy
3. NLTK
4. Keras

### What is a Conversational Chat Bot

In the Essence of the world, it is a robot, that enables a machine to simulate human like conversations.
To achieve this, the user interface needs to be as humanlike and conversational as possible. A conversational chatbot must understand the users intents and how to respond to the user.

Fundamentally a chatbot turns raw data into conversation, The two keys bits of data that a chatbot needs to process are what people are saying to it and what it needs to respond. The easiet example to understand is a simple customer service chat Bot.

__If you intend to build a chatbot for a Customer or a Company you would need to understand the customer/company workflow, customer service and needs and Data, Beacuse Data is the key to develop a truly conversational chatbot__

#### Importing the Libraries to be used 

In [ ]:
import json
import numpy as np 
import nltk 
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

Using TensorFlow backend.


#### Opening our Json File

In [ ]:
with open("chat.json") as file:
    bot = json.load(file)

### Looping through the opened Json file and appending it to different list for further iteration

In [ ]:
words = []
labels = []
intent_part_x = []
intent_part_y = []


for intent in bot["intents"]:
    for pattern in intent["patterns"]:
        token = nltk.wordpunct_tokenize(pattern) #Tokenizing the pattern
        #Since token is a list no need of appending we just use the function extend
        words.extend(token)
        intent_part_x.append(token)
        intent_part_y.append(intent["tag"])


    if intent["tag"] not in labels:
        labels.append(intent["tag"])

### Stemming the Word using NLTK Stemmer and Sorting it Alphabetically

In [ ]:
words = [ps.stem(w.lower()) for w in words]

words = sorted(list(set(words)))

labels = sorted(labels)

#### Creating The Train and Output List

In [ ]:
train = []
output = []

In [ ]:
# Creating the Bag of word list from the labels list

out_empty = [0 for i,col in enumerate(labels)]

In [ ]:
#Intent_part_x is the list that contains all the available Input for the user in the Json File

#Iterating through the file to create a list of bag of words to check if a user Input was present in the Json File

#If present then append 1

for x, doc in enumerate(intent_part_x):
    bow = []

    token = [ps.stem(w) for w in doc]

    for w in words:
        if w in token:
            bow.append(1)
        else:
            bow.append(0)

    output_row = out_empty[:]
    output_row[labels.index(intent_part_y[x])] = 1

    train.append(bow)  #Creating the Train 
    output.append(output_row) #Creating the output list

In [ ]:
#Concverting it to an numpy array since deep learning deals with array

train = np.array(train)
output = np.array(output)

In [ ]:
print("Train Array")

print(train[5])


print(f"\nLength of train array: {len(train[5])}")

## Building our Deep Learning Neural Net

### We would using only Three Layer since it is not a complex File 

#### We would be using Relu activation function and The Softmax activation since we are expected to have multiple outcome

In [ ]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(output[0]), activation='softmax'))

In [ ]:
#We would be using the Stochastic Gradient Descent and the Nesterov Accelerated Gradient Descent

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [ ]:
#Training the Model 
model.fit(train, output, epochs=200, batch_size=8, verbose=1)

### Creating the Functions that would enable the user interact with the chat bot runs

In [ ]:
def bag_of_words(s,words):
    """
    Input:
    This functions create a bag of word for the sentence the user enters
    
    Output:
    It returns a numpy array to be used 
    Note: The Numpy array was reshaped to an array of 79 because the length of our train[0] is 79
    If you make changes or edit the json file to suit your needs then the Length of your train will change
    Do well to play around the codes, make changes to suit your need and understand how it works
    """
    bag = [ 0 for i in range(len(words))]

    s_words_token = nltk.wordpunct_tokenize(s)
    s_words_token = [ps.stem(word.lower()) for word in s_words_token]

    for se in s_words_token:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
    
    return np.array(bag).reshape(-1,79)

In [ ]:
def chat():
    """
    Input:
    This is function that Activate the ChatBot for the User to Interact with it
    
    Output:
    It gives a Responses randomly.
    We selected a threshold to prevent it from giving out irrelevatnt response to the user
    You can quit talking with the Chat Bot by type quit in the chat Column
    
    Note: The Threshold was not selected Randomly, it was done after sereies of conversation with the bot to know,
    which threshold it starts giving out irrelevant responses if the responses is not in it database
    """
    print("Start Talking with the Bot,To Stop type quit")
    while True:
        inp = input("You :   ")
        if inp.lower() == "quit":
            break 

        input_data = [bag_of_words(inp, words)]
        results = model.predict(input_data)[0]
        results_index = np.argmax(results)
        tag = labels[results_index]
        
        #To see how the model perfoms on data that are not in the database
        #print(f"Model Prediction: {results[results_index]}")
        
        if results[results_index] >= 0.794:
            #Looping through the json file
            for tags in bot["intents"]:
                if tags["tag"] == tag:
                    responses = tags["responses"]
            print(np.random.choice(responses))
        
        else:
            print("I dont quite Understand, Ask another question")

In [ ]:
chat()

### Conclusion…

Building a ChatBot is not Rocket Science you can Build a chatbot for yourself and integrate it to your website, applications,or other social media platform to help keeps your conversation on the go even if you are not online.

You can fork out this repo and work on it to suit your own need things you need to do is edit the Json File to how best you want your chatbot to interact with the user.

As your Responses, Tag, Intents increase you would also have to increase the the layers in your model to have a better accuracy and responses for the chatbot.

You can read about about the __Nesterov Accelerated Gradient Descent__

__Natural Language Processing__ is an exciting Field that is fast growing you can also try other Deep Learning Framework like Fast.ai, PyTorch or Theano to see how your data learn and how the chatbot works.


#### Connect with me on [twitter](https://www.twitter.com/imonemmanuel).

#### Connect with me on [linkedin](https://www.linkedin.com/in/imonemmanuel).